# **Segmentación Semántica Multimodal: Qué hay en la Imagen**

**Computación visual**

Gabriela Gallegos Rubio

In [ ]:
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install supervision opencv-python matplotlib
# Descargar el checkpoint de SAM
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -O sam_vit_h_4b8939.pth

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-kv08mxca
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-kv08mxca
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
--2025-05-01 01:34:03--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.173.132.17, 18.173.132.81, 18.173.132.13, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.173.132.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   257MB/s    in 29s     

2025-05-01 01:34:32 (83.8 MB/s) - ‘sam_vit_h_4b8939.pth’ saved [2564550879/2564550879]



In [4]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from segment_anything import SamPredictor, sam_model_registry
import imageio
from PIL import Image
import torch

In [5]:
# Crear directorios para la entrega
base_dir = "2025-04-29_taller_segmentacion_semantica_sam_deeplab"
os.makedirs(f"{base_dir}/imagenes_entrada", exist_ok=True)
os.makedirs(f"{base_dir}/mascaras_salida", exist_ok=True)
os.makedirs(f"{base_dir}/resultados", exist_ok=True)

# Cargar imagen
image_path = f"{base_dir}/imagenes_entrada/jirafa.png"
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"No se pudo cargar la imagen en {image_path}")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Cargar modelo
checkpoint_path = "sam_vit_h_4b8939.pth"  # Descarga desde: https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
if not os.path.exists(checkpoint_path):
    raise FileNotFoundError(f"No se encontró el checkpoint en {checkpoint_path}. Descarga 'sam_vit_h_4b8939.pth' desde el enlace proporcionado.")
sam = sam_model_registry["vit_h"](checkpoint=checkpoint_path).to("cpu")  # Usar CPU
predictor = SamPredictor(sam)
predictor.set_image(image)

# Seleccionar punto para segmentar
input_point = np.array([[300, 300]])  # Punto en la imagen
input_label = np.array([1])

# Obtener máscaras
masks, scores, _ = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=True
)

# Definir colores para cada máscara
colors = ['red', 'green', 'blue']  # Un color por máscara (hasta 3 máscaras)

# Función para dibujar contornos de la máscara
def draw_mask_contours(image, mask, ax, color='red'):
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        contour = contour.squeeze()
        if contour.ndim == 1:
            continue
        ax.plot(contour[:, 0], contour[:, 1], color=color, linewidth=2)

# Función para redimensionar y convertir a RGB
def prepare_image_for_gif(image_array, target_shape):
    image_pil = Image.fromarray(image_array)
    image_pil = image_pil.resize((target_shape[1], target_shape[0]), Image.LANCZOS)
    # Convertir a RGB (3 canales)
    if image_pil.mode != 'RGB':
        image_pil = image_pil.convert('RGB')
    return np.array(image_pil)

# Visualizar resultados, guardar máscaras y preparar imágenes para GIF
gif_frames = []
target_shape = image.shape[:2]  # Altura y ancho de la imagen original

# Añadir imagen original al GIF
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.title("Imagen Original")
plt.axis("off")
original_path = f"{base_dir}/resultados/imagen_original.png"
plt.savefig(original_path, bbox_inches='tight')
plt.close()
original_frame = imageio.imread(original_path)
original_frame = prepare_image_for_gif(original_frame, target_shape)
gif_frames.append(original_frame)
print(f"Original frame shape: {original_frame.shape}")

# Procesar cada máscara
for i, (mask, score) in enumerate(zip(masks, scores)):
    # Guardar máscara como imagen binaria
    mask_binary = mask.astype(np.uint8) * 255
    mask_pil = Image.fromarray(mask_binary)
    mask_binary_path = f"{base_dir}/mascaras_salida/mask_{i}.png"
    mask_pil.save(mask_binary_path)

    # Visualizar máscara con color específico
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    draw_mask_contours(image, mask, plt.gca(), color=colors[i % len(colors)])
    plt.title(f"Máscara {i} (Score: {score:.3f})")
    plt.axis("off")
    mask_colored_path = f"{base_dir}/resultados/mask_colored_{i}.png"
    plt.savefig(mask_colored_path, bbox_inches='tight')
    plt.close()

    # Añadir imágenes al GIF: coloreada y binaria
    colored_frame = imageio.imread(mask_colored_path)
    colored_frame = prepare_image_for_gif(colored_frame, target_shape)
    gif_frames.append(colored_frame)
    print(f"Colored frame {i} shape: {colored_frame.shape}")

    binary_frame = imageio.imread(mask_binary_path)
    binary_frame = prepare_image_for_gif(binary_frame, target_shape)
    gif_frames.append(binary_frame)
    print(f"Binary frame {i} shape: {binary_frame.shape}")

# Guardar imagen original
Image.fromarray(image).save(original_path)

# Generar GIF animado
gif_path = f"{base_dir}/resultados/segmentation_process.gif"
imageio.mimsave(gif_path, gif_frames, duration=1000)  # 1 segundo por frame

<ipython-input-5-0dbb09a92e65>:66: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  original_frame = imageio.imread(original_path)


Original frame shape: (861, 1304, 3)


<ipython-input-5-0dbb09a92e65>:90: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  colored_frame = imageio.imread(mask_colored_path)
<ipython-input-5-0dbb09a92e65>:95: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  binary_frame = imageio.imread(mask_binary_path)


Colored frame 0 shape: (861, 1304, 3)
Binary frame 0 shape: (861, 1304, 3)
Colored frame 1 shape: (861, 1304, 3)
Binary frame 1 shape: (861, 1304, 3)
Colored frame 2 shape: (861, 1304, 3)
Binary frame 2 shape: (861, 1304, 3)
